# Module 6: Markov Chains

A markov chain is a system whose evolution is described by a stochastic process.
If the structure of such process is that dependence of current state with respect to entire past
is completely captured by the dependence on the last sample, also know as the **Markov property**,
then this process is a **Markov Chain**.

The Markov property is mathetically expressed by the following [conditional independence](https://en.wikipedia.org/wiki/Conditional_independence):

$$ P(X_{n+1}|X_n, X_{n-1}, ... X_1) = P(X_{n+1}|X_n) $$

That is, the probability of next state within a process is the governed only by the current state.

In this lab we will cover:
* Transition probabilities of the Markov Chain
* Ergodicity
* Stationary distribution

Here's more information on Markov Chains and live demos:
http://setosa.io/ev/markov-chains/


In [ ]:
import os, sys
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf

### Note: NumPy Linear Algebra Package

This notebook uses the NumPy Linear Algebra package.
It may be useful to revisit this documentation at a later date.

This is necessary because we are getting into some more _fun_ math.
The goal remains understanding concepts related to this tool and the type of 
data that can benefit from these types of models.

You are encouraged to skip the math and come back to it later if you feel the need for an adventure.


In [ ]:
help(np.linalg)

# Example problem

It's perhaps best to start with an example.
![Markov Chain Example](../resources/mc1.png)

Consider a radio station playing two genres of songs, which we label as 0 and 1.

From time to time the radio station may switch genre or keep playing songs from the same genre.
This graph gives the probabilty of whether such a switch takes place at any 
given time based on the current genre being played.

These transition probabilities can be collected in a matrix, known as the transition matrix.

$$P_{ij} = P(\text{radio station switches genre from i to j}) $$

The Markov Chain can be used to anwser questions such as:
* What's the probabilty of current state at any moment? (Or likelihood of each genre being played?)
  * This is asking for the **stationary distribution**
* What's the average time for the system to go back to each state? (Or how often does it play the same genre again?)

To answer these questions, 
it's necessary to first realize that it's possible that some states could only occur
a finite number of times i.e. there may be states that don't recur 
(usually due to topology of the graph).

Here we only study the more generally meaningful case where

$$ P(\text{ever return to state i})=1 $$



A markov chain is said to be **ergodic** when we can substitute time averages for ensemble averages:

$$ \lim _ {k \to \infty } {\frac{1}{\frac{1}{k} \sum_{k'=1}^{k}{T_i(k')} }} = \frac {1}{E[T_i(k)]}$$

where $T_i(k)$ is time elapses between (k-1)-th and k-th return to state i.

The **ergodicity theorem** states that

1. If a **stationary distribution** $\pi$ exists for a Markov Chain, it's ergodic.
2. Such **stationary distribution** is independent of initial distribution $\pi_0$ of Markov Chain if it's ergodic.

Therefore, assuming existence of a stationary distribution of the Markov Chain,
the solution can be found using the **power method**:

$$ \pi = \lim_{n \to \infty }{\pi_0 P^{n}}$$



Back to the original problem, suppose the radio station has 1/3 probability 
of switching genre and 2/3 probability of staying in the same genre. 

Base on the fact that this is symmetric with respect to each genre, 
we should intuitively have probability of tuning into each genre equal to 1/2.

Now let's solve using the power method and verify.

In [ ]:
# Transition matrix
P = np.array([[2/3, 1/3],
              [1/3, 2/3]])

# Initial distribution can be anything that sums up to 1
pi0 = np.array([0.5, 0.5])

# Compute stationary distribution - power method
np.dot(pi0, np.linalg.matrix_power(P, 50))

We see that the probability of state 0 and 1 are both 0.5.

But what if probability of switching from 0 to 1 is 1/4 and probability of switching from 1 to 0 is 1/3,
so that this radio station baises towards one genre?

We now can answer this less trivial question as well with the **power method**.

In [ ]:
# Transition matrix
P = np.array([[3/4, 1/4], 
              [1/3, 2/3]])

# Initial distribution can be anything that sums up to 1
pi0 = np.array([0.5, 0.5])

# Compute stationary state - power method
np.dot(pi0, np.linalg.matrix_power(P, 50))

We see a new probability of each genre given the changed transition probabilities.
Intuitively, we are less likely to switch away from genre `0` to `1` than vice versa, 
and therefore more likely to hear a song from genre 0.


Another tool for solving this problem is **eigen-decomposition** due to [Perron-Frobenius theorem](https://en.wikipedia.org/wiki/Perron%E2%80%93Frobenius_theorem).

Let's compare the results:

In [ ]:
# Some random 5x5 transition matrix
P = np.random.rand(5, 5)
P /= P.sum(axis=1)[:,np.newaxis] # normalization along axis 1

# Compute stationary state - power method
pi0 = np.random.rand(5)
pi0 /= pi0.sum()
a = np.dot(pi0, np.linalg.matrix_power(P, 50))
print(a)

# Compute stationary state - eigen decomposition
L, Q = np.linalg.eig(P.T)
# Pick eigenvector whose corresponding eigenvalue is closest to 1
b = Q[:,np.argmin(abs(L - 1.0))].real
# Normalize into a probability distribution
b /= b.sum()
print(b)

np.allclose(a,b)

Here's an implementation of the power method with **TensorFlow**.
Currently neither matrix power or eigen-decomposition is supported,
but we may still want to leverage TensorFlow's high scalibility to process datasets.

This algorithm will use [divide and conquer](https://en.wikipedia.org/wiki/Divide_and_conquer_algorithm)
strategy to reduce time complexity as well as
space complexity for representing such computation in TensorFlow.

The advantage is that this program will utilize GPU when available,
and you wouldn't need to change anything to let that happen.

In [ ]:
# Compute stationary state - power method
def mat_power(M, n):
    """ Construct a graph that raises square matrix M to n-th power where n>=1
    This generates a computational graph with space complexity O(log(n)).
    """
    assert n>=1
    # trivial cases
    if n==2:
        return tf.matmul(M, M)
    elif n==1:
        return M
    
    # divide & conquer
    A = mat_power(M, n//2)
    A2 = tf.matmul(A, A)
    if n&1: # odd power
        return tf.matmul(A2, M)
    else: # even power
        return A2

def get_stationary_state(P):
    pi0 = tf.constant(np.ones((1, len(P)))/len(P))
    transition_matrix = tf.constant(P)
    stationary_state = tf.squeeze(tf.matmul(pi0, mat_power(transition_matrix, 50)))
    with tf.Session() as sess:
        return sess.run(stationary_state)

a = get_stationary_state(P)
print(a)

np.allclose(a,b)

# Save your Notebook

Now... read this!
  * https://flowingdata.com/2015/12/15/a-day-in-the-life-of-americans/